# RAG Chat-Bot Task

# Summary

## This script sets up a Streamlit-based chatbot application using PDF documents as the knowledge base. It involves several key steps:

## Import Libraries: 
Necessary libraries are imported for document loading, text splitting, environment variable access, Pinecone vector database, embedding, chat model, retrieval-based QA, and Streamlit web app.

## Load Environment Variables: 
API keys for Pinecone and OpenAI are loaded from a .env file.

## Document Pre-Processing:
PDF documents from the "document/" directory are loaded and split into chunks of 1000 characters.

## Embedding Database: 
A Pinecone index is created if it doesn't exist, and the split documents are embedded using OpenAI's embeddings. This is cached using Streamlit's st.cache_resource for efficiency.

## Initialize Models: 
The ChatOpenAI model is instantiated and the embedding database is created.

## Retrieve Answers: 
A function is defined to retrieve answers to user queries using a retrieval-based question-answering chain.

## Streamlit App: 
The main function sets up the Streamlit web app, which manages chat history, accepts user input, and displays responses. The app title is set to "Abhi-Alemeno RAG CHAT-BOT".


## Use VS Code to eun main file "main.py" which is in same directory to launch streamlit app, in jupyter notebook it won't run efficiently...Thank You

# Important Note
I sincerely apologize for deviating from the initial plan of using local embedding models and LLM (Language Model). Unfortunately, my laptop's low specifications made it exceedingly challenging to efficiently run both the local embedding models and the LLM simultaneously. This limitation hindered my ability to implement the project as originally intended.

Given a more suitable environment or the oppertunity, I am confident that I can successfully complete the task according to the specified requirements. I appreciate your understanding and the opportunity to address this issue.

In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_community.vectorstores import Pinecone as PineconeVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import streamlit as st
from dotenv import load_dotenv

C:\Users\ABHIMANYU\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Imports necessary libraries for document loading, text splitting, environment variable access, Pinecone database, embedding, chat model, retrieval-based QA, and Streamlit web app.

In [2]:
load_dotenv()

True

Loads environment variables from a .env file.

In [3]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

This snippet retrieves the Pinecone and OpenAI API keys, along with the Pinecone environment variable, from the environment and sets the OpenAI API key in the environment variables. This ensures secure access to the APIs.

In [4]:
# Document Pre-Processing
def doc_processing():
    loader = PyPDFDirectoryLoader("document/", glob='**/*.pdf')
    docs = loader.load()
    text_splitter = CharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0
    )
    docs_split = text_splitter.split_documents(docs)
    return docs_split

This function processes PDF documents:

Loads all PDF files from the specified directory.
Splits the loaded documents into smaller chunks of 1000 characters without overlap.
Returns the split documents for further processing.

In [5]:
# Embedding DB
@st.cache_resource
def embedding_db():
    embeddings = OpenAIEmbeddings()
    pc = Pinecone(api_key=PINECONE_API_KEY)

    if 'abhi-alemeno' not in pc.list_indexes().names():
        pc.create_index(
            name='abhi-alemeno',
            dimension=1536,  # Use 1536 dimensions for text-embedding-ada-002
            metric='cosine',  # Use cosine similarity
            spec=ServerlessSpec(
                cloud='aws',
                region=PINECONE_ENV
            )
        )

    doc_split = doc_processing()
    vectorstore = PineconeVectorStore.from_documents(doc_split, embedding=embeddings, index_name='abhi-alemeno')
    return vectorstore

This cached function sets up the embedding database:
    
Initializes OpenAI embeddings.
Connects to Pinecone using the API key.
Creates a Pinecone index if it doesn't already exist, specifying the dimension and similarity metric.
Processes the documents and creates a Pinecone vector store from the split documents.
Returns the vector store for use in retrieval tasks.

In [10]:
llm = ChatOpenAI()  # Instantiate the ChatOpenAI model
doc_db = embedding_db()

This snippet initializes the language model (ChatOpenAI) and creates the embedding database by calling the embedding_db function, setting up the environment for retrieval-based question answering.

In [7]:
def retrieval_answer(query):
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=doc_db.as_retriever(),
    )
    result = qa.run(query)
    return result

This function retrieves answers to user queries:

Uses the RetrievalQA chain with the language model and document retriever.
Runs the query through the QA chain.
Returns the result, which is the answer to the query.

In [12]:
def main():
    st.title("Abhi-Alemeno RAG CHAT-BOT")
    
    # Initialize chat history in session state if not present
    if "messages" not in st.session_state:
        st.session_state["messages"] = []
    
    # Display chat messages from history on app rerun
    for message in st.session_state["messages"]:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # Accept user input using chat input widget
    prompt = st.chat_input("Ask Your Query...")
    
    if prompt:
        # Add user message to chat history
        st.session_state["messages"].append({"role": "user", "content": prompt})
        
        # Display user message in chat
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Retrieve and display assistant response
        answer = retrieval_answer(prompt)
        st.session_state["messages"].append({"role": "assistant", "content": answer})
        
        with st.chat_message("assistant"):
            st.markdown(answer)
        
if __name__ == "__main__":
    main()

This function sets up and runs the Streamlit web application:
    
Sets the title of the web app.
Initializes chat history in the session state.
Displays chat messages from history on app rerun.
Accepts user input via a chat input widget.
Adds user messages to chat history and displays them.
Retrieves answers to user queries and displays them.
Runs the main function if the script is executed directly, starting the Streamlit app.